<a href="https://colab.research.google.com/github/ralph27/ZAKA-hands-on/blob/master/Fine_Tuning_BERT_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT with LoRA
---
© 2023, Zaka AI, Inc. All Rights Reserved.


**Objective:** In this practical exercise, we will the LoRA fine-tuning approach. Our main objective is to fine-tune a BERT model to categorize movie ratings binarily. The dataset consists of polarized ratings between positive and negative reviews and our goal is to increase the model's accuracy with fine-tuning



## Importing Needed Packages

### Prerequisite Libraries:

1. **transformers**: Hugging Face transformers library, which is used for working with pre-trained transformer-based models, such as BERT.
2. **peft**: Library that allows the usage of PEFT techniques such as LoRA.
3. **evaluate**: A library that makes evaluating and comparing models and reporting their performance easier and more standardized.

In [ ]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


## Dataset Loading

The selected dataset for this Notebook is the **IMDB** dataset. The decision to use this dataset is based on two reasons.
- First, the dataset offers a substantial amount of data, providing a rich resource for training machine learning models effectively.
- Second, the dataset's straightforward columns are well-suited for the purpose of comparing various machine learning algorithms.


Let's start by downlaoding and loading the dataset:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Data Preprocessing

### Tokenization
Tokenization will allow us to feed batches of sequences at the same time:
- SMS need to be padded to the same length
- SMS need to be truncated to match the model's maximum input length.

To perform the tokenization of the data, we also need to choose a pre-trained tokenizer. For our case, the basic model (bert-base-cased) will be sufficient:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Splitting dataset into Train and Test Sets
It is important to split the dataset into train and test sets while preserving the distribution of spam and ham texts. In our case this is already done with the previously set Tokenizer, so we are only going to fetch the train and test splits, and set the same seed for both so it's reproducible.

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

### Defining LoRA Configurations
Since we are using the peft library, we can set a LoRA configuration to be loaded later on, without much boiler-plate code.

r=  the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.

lora_alpha= responsible for the scaling of the weight matrices

lora_dropout= dropout techniques that avoids overfitting

In [ ]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

### Loading the pre-trained BERT
Now we will load the pre-trained BERT, specifically for Sequence Classification, and set the number of labels to 2 since this is our goal

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Adding LoRA to our Model
To add the Lora configurations we set earlier to our model, we use the get_peft_model function, passing to it our model and the lora_config variable we set earlier.

In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

## Creating our Metrics Function
We will create a compute_metrics function during this section that will be used during Training. We will evaluate our model based on accuracy, and this function will be passed onto our Trainer( ) class that will train our model with the LoRA configurations.

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Training our Model with LoRA
Finally, we will train our model and see how the accuracy enhances through each epoch. The Trainer( ) class will allow seamless implementation of our training process, by just adding the previous variables and functions we created earlier

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=25,)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.679898,0.595000
2,No log,0.672862,0.606000
3,No log,0.662353,0.629000
4,0.693400,0.643180,0.664000
5,0.693400,0.616434,0.686000
6,0.693400,0.580412,0.703000
